<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here

summary = pd.DataFrame({
    "dtype": df.dtypes,
    "non_null_count": df.notnull().sum(),
    "missing_count": df.isnull().sum(),
    "missing_percent": (df.isnull().mean() * 100).round(2)
})

print(summary)

                       dtype  non_null_count  missing_count  missing_percent
ResponseId             int64           65437              0             0.00
MainBranch            object           65437              0             0.00
Age                   object           65437              0             0.00
Employment            object           65437              0             0.00
RemoteWork            object           54806          10631            16.25
...                      ...             ...            ...              ...
JobSatPoints_11      float64           29445          35992            55.00
SurveyLength          object           56182           9255            14.14
SurveyEase            object           56238           9199            14.06
ConvertedCompYearly  float64           23435          42002            64.19
JobSat               float64           29126          36311            55.49

[114 rows x 4 columns]


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here

stats = df.describe()

print(stats)

         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000         0.00000   
25%          0.000000 

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here

# Look at unique values in the Country column
print(df["Country"].unique())

# Find rows where Country has non-letter characters (likely inconsistent)
mask = ~df["Country"].str.match(r'^[A-Za-z\s]+$', na=False)
print(df.loc[mask, "Country"])

['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 'Norway'
 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria' 'Switzerland'
 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia' 'Brazil'
 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France' 'Saudi Arabia'
 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden' 'Netherlands' 'Croatia'
 'Pakistan' 'Czech Republic' 'Republic of North Macedonia' 'Finland'
 'Slovakia' 'Russian Federation' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'United Republic of Tanzania' 'Hungary' 'Argentina' 'Portugal'
 'Sri Lanka' 'Latvia' 'China' 'Singapore' 'Lebanon' 'Spain' 'South Africa'
 'Lithuania' 'Viet Nam' 'Dominican Republic' 'Indonesia' 'Kosovo'
 'Morocco' 'Taiwan' 'Georgia' 'San Marino' 'Tunisia' 'Bangladesh'
 'Nigeria' 'Liechtenstein' 'Denmark' 'Ecuador' 'Malaysia' 'Albania'
 'Azerbaijan' 'Chile' 'Ghana' 'Peru' 'Bolivia' 'Egypt' 'Luxembourg'
 'Montenegro' 'Cyprus' 'Paraguay' 'Kazakhstan' 'Slovenia' 'Jordan'
 

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
## Write your code here


country_mapping = {
    'United States of America': 'USA',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Iran, Islamic Republic of...': 'Iran',
    'Republic of North Macedonia': 'North Macedonia',
    'Russian Federation': 'Russia',
    'United Republic of Tanzania': 'Tanzania',
    'Viet Nam': 'Vietnam',
    'Venezuela, Bolivarian Republic of...': 'Venezuela',
    'Republic of Korea': 'South Korea',
    'Hong Kong (S.A.R.)': 'Hong Kong',
    "Lao People's Democratic Republic": 'Laos',
    'Democratic Republic of the Congo': 'DRC',
    'Syrian Arab Republic': 'Syria',
    "Côte d'Ivoire": 'Ivory Coast',
    'Congo, Republic of the...': 'Congo',
    'Democratic People\'s Republic of Korea': 'North Korea',
    'Libyan Arab Jamahiriya': 'Libya',
    'Brunei Darussalam': 'Brunei',
    'Micronesia, Federated States of...': 'Micronesia'
}

# 1. Handle missing values first to avoid errors during mapping
df['Country'] = df['Country'].fillna('Unspecified')

# 2. Apply the mapping to standardize the entries
df['Country'] = df['Country'].replace(country_mapping)

# 3. Standardize capitalization for consistency
df['Country'] = df['Country'].str.title()

# Verify the changes by checking the unique values again
print(df['Country'].unique())

['Usa' 'United Kingdom' 'Canada' 'Norway' 'Uzbekistan' 'Serbia' 'Poland'
 'Philippines' 'Bulgaria' 'Switzerland' 'India' 'Germany' 'Ireland'
 'Italy' 'Ukraine' 'Australia' 'Brazil' 'Japan' 'Austria' 'Iran' 'France'
 'Saudi Arabia' 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden'
 'Netherlands' 'Croatia' 'Pakistan' 'Czech Republic' 'North Macedonia'
 'Finland' 'Slovakia' 'Russia' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'Tanzania' 'Hungary' 'Argentina' 'Portugal' 'Sri Lanka' 'Latvia' 'China'
 'Singapore' 'Lebanon' 'Spain' 'South Africa' 'Lithuania' 'Vietnam'
 'Dominican Republic' 'Indonesia' 'Kosovo' 'Morocco' 'Taiwan' 'Georgia'
 'San Marino' 'Tunisia' 'Bangladesh' 'Nigeria' 'Liechtenstein' 'Denmark'
 'Ecuador' 'Malaysia' 'Albania' 'Azerbaijan' 'Chile' 'Ghana' 'Peru'
 'Bolivia' 'Egypt' 'Luxembourg' 'Montenegro' 'Cyprus' 'Paraguay'
 'Kazakhstan' 'Slovenia' 'Jordan' 'Venezuela' 'Costa Rica' 'Jamaica'
 'Thailand' 'Nicaragua' 'Myanmar' 'South Korea' 'Rwanda'
 'Bosnia And Herzegovina' 'Benin' 'El 

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
## Write your code here

# One-hot encode the Employment column
employment_encoded = pd.get_dummies(df["Employment"], prefix="Employment")

# Join back to original dataframe (optional)
df = pd.concat([df, employment_encoded], axis=1)

print(df.head())

   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
## Write your code here

missing_values = df.isnull().sum()

# Sort the results in descending order to see the highest counts first
columns_with_most_missing = missing_values.sort_values(ascending=False)

print("Columns with the highest number of missing values:")
print(columns_with_most_missing)

Columns with the highest number of missing values:
AINextMuch less integrated                                                                                                                               64289
AINextLess integrated                                                                                                                                    63082
AINextNo change                                                                                                                                          52939
AINextMuch more integrated                                                                                                                               51999
EmbeddedAdmired                                                                                                                                          48704
                                                                                                                                                         .

<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [11]:
## Write your code here


# Step 1: Convert to numeric (coerce any non-numeric values to NaN)
df["AINextMuch less integrated"] = pd.to_numeric(
    df["AINextMuch less integrated"], errors="coerce"
)

# Step 2: Compute mean or median
mean_value = df["AINextMuch less integrated"].mean()
median_value = df["AINextMuch less integrated"].median()

# Step 3: Impute missing values (choose mean or median)
# Using mean
df["AINextMuch less integrated_mean"] = df["AINextMuch less integrated"].fillna(mean_value)

# Using median
df["AINextMuch less integrated_median"] = df["AINextMuch less integrated"].fillna(median_value)

# Step 4: Verify
print("Missing values after mean imputation:", df["AINextMuch less integrated_mean"].isnull().sum())
print("Missing values after median imputation:", df["AINextMuch less integrated_median"].isnull().sum())


Missing values after mean imputation: 65437
Missing values after median imputation: 65437


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [12]:
## Write your code here

# Categorical column
cat_col = "Employment_Student, full-time;Student, part-time;Employed, part-time"

# Step 1: Count missing values
missing_before = df[cat_col].isnull().sum()
print(f"Missing values before imputation: {missing_before}")

# Step 2: Find mode
mode_value = df[cat_col].mode()[0]
print(f"The most frequent value is: {mode_value}")

# Step 3: Impute missing values
df[cat_col] = df[cat_col].fillna(mode_value)

# Step 4: Verify
missing_after = df[cat_col].isnull().sum()
print(f"Missing values after imputation: {missing_after}")


Missing values before imputation: 0
The most frequent value is: False
Missing values after imputation: 0


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [13]:
## Write your code here

# Step 1: Impute missing values with the median.
# This ensures that your min and max calculations are accurate.
median_comp = df['ConvertedCompYearly'].median()
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(median_comp)

# Step 2: Calculate the minimum and maximum values.
min_val = df['ConvertedCompYearly'].min()
max_val = df['ConvertedCompYearly'].max()

# Step 3: Apply the Min-Max Scaling formula to the column.
df['ConvertedCompYearly_MinMax'] = (df['ConvertedCompYearly'] - min_val) / (max_val - min_val)

# Step 4: Display the first 5 rows of the original and new columns to verify.
print("Original and Min-Max Normalized Columns:")
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_MinMax']].head())

Original and Min-Max Normalized Columns:
   ConvertedCompYearly  ConvertedCompYearly_MinMax
0              65000.0                    0.003998
1              65000.0                    0.003998
2              65000.0                    0.003998
3              65000.0                    0.003998
4              65000.0                    0.003998


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [14]:
## Write your code here

import math

# Log-transform ConvertedCompYearly
df["ConvertedCompYearly_log"] = df["ConvertedCompYearly"].apply(
    lambda x: math.log1p(x) if pd.notnull(x) else x
)

# Check the result
print(df[["ConvertedCompYearly", "ConvertedCompYearly_log"]].head())

   ConvertedCompYearly  ConvertedCompYearly_log
0              65000.0                11.082158
1              65000.0                11.082158
2              65000.0                11.082158
3              65000.0                11.082158
4              65000.0                11.082158


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [15]:
## Write your code here

# Convert YearsCodePro to numeric, coercing errors to NaN, then fill NaN with 0
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce').fillna(0)

# Function to categorize experience level
def categorize_experience(years):
    if years <= 2:
        return 'Entry-level'
    elif 3 <= years <= 9:
        return 'Mid-level'
    elif years >= 10:
        return 'Senior-level'
    return None  # instead of np.nan

# Apply the function to create ExperienceLevel
df['ExperienceLevel'] = df['YearsCodePro'].apply(categorize_experience)

# Verify results
print(df['ExperienceLevel'].value_counts())

ExperienceLevel
Entry-level     23540
Senior-level    21661
Mid-level       20236
Name: count, dtype: int64


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
